<a href="https://colab.research.google.com/github/gizemkaratas/Musteri-Yasam-Boyu-Degeri-Tahmini-CLTV-/blob/main/CLTV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BG-NBD ve Gamma-Gamma ile CLTV Prediction**

In [28]:
!pip install lifetimes
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

In [29]:
df=pd.read_excel("/content/online_retail_II.xlsx",sheet_name="Year 2010-2011")

# **Veri Önişleme**

In [30]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


# Değişkenler

InvoiceNo: Fatura numarası. Her işleme yani faturaya ait eşsiz numara. C ile başlıyorsa iptal edilen işlem.
StockCode: Ürün kodu. Her bir ürün için eşsiz numara.
Description: Ürün ismi
Quantity: Ürün adedi. Faturalardaki ürünlerden kaçar tane satıldığını ifade etmektedir.
InvoiceDate: Fatura tarihi ve zamanı.
UnitPrice: Ürün fiyatı (Sterlin cinsinden)
CustomerID: Eşsiz müşteri numarası
Country: Ülke ismi. Müşterinin yaşadığı ülke.

In [31]:
df.shape

(541910, 8)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [33]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,541910.0000,9.5522,-80995.0000,1.0000,3.0000,10.0000,80995.0000,218.0810
InvoiceDate,541910,2011-07-04 13:35:22.342307584,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
Price,541910.0000,4.6111,-11062.0600,1.2500,2.0800,4.1300,38970.0000,96.7598
Customer ID,406830.0000,15287.6842,12346.0000,13953.0000,15152.0000,16791.0000,18287.0000,1713.6031


In [34]:
df.isnull().sum()

,0
Invoice,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
Price,0
Customer ID,135080
Country,0


In [35]:
# boş değerleri silelim

df.dropna(inplace=True)


In [36]:
# dfye TotalPrice değişkeni ekleyelim

df["TotalPrice"] = df["Quantity"] * df["Price"]


In [37]:
# Invoice no da C olan iadeleri silelim

df = df[~df["Invoice"].str.contains("C", na=False)]


In [38]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397925.0000,13.0218,1.0000,2.0000,6.0000,12.0000,80995.0000,180.4200
InvoiceDate,397925,2011-07-10 23:44:09.817126400,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397925.0000,3.1162,0.0000,1.2500,1.9500,3.7500,8142.7500,22.0968
Customer ID,397925.0000,15294.3086,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000,1713.1727
TotalPrice,397925.0000,22.3947,0.0000,4.6800,11.8000,19.8000,168469.6000,309.0552


In [39]:
df = df[df["Quantity"] > 0]


In [40]:
# prompt: price değişkeninden 0 ve 0dan küçükleri sil

df = df[df["Price"] > 0]


In [41]:
df.dtypes

,0
Invoice,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
Price,float64
Customer ID,float64
Country,object
TotalPrice,float64


In [42]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0000,12.9882,1.0000,2.0000,6.0000,12.0000,80995.0000,179.3316
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0000,3.1165,0.0010,1.2500,1.9500,3.7500,8142.7500,22.0979
Customer ID,397885.0000,15294.4169,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000,1713.1444
TotalPrice,397885.0000,22.3970,0.0010,4.6800,11.8000,19.8000,168469.6000,309.0707


In [43]:
#Aykırı değerleri baskılayalım
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit

In [44]:
replace_with_thresholds(df, "Quantity")


In [45]:
replace_with_thresholds(df, "Price")

In [46]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0000,11.8308,1.0000,2.0000,6.0000,12.0000,298.5000,25.5231
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0000,2.8935,0.0010,1.2500,1.9500,3.7500,37.0600,3.2272
Customer ID,397885.0000,15294.4169,12346.0000,13969.0000,15159.0000,16795.0000,18287.0000,1713.1444
TotalPrice,397885.0000,22.3970,0.0010,4.6800,11.8000,19.8000,168469.6000,309.0707


# **BG-NBD ile Ortalama Satınalma Tahminleme**

## **Modelin Kurulması İçin Gereken Değişkenler**

*   recency= müşterinin son satınalma tarihi-ilk satınalma tarihi
*   T= analiz tarihi - müşterinin ilk alışveriş yaptığı tarih, müşterinin yaşıdır haftalığa dönüştürülmelidir
*   frequency= tekrar eden satış sayısı, 1 den büyük olmalı, müşterinin toplam satış sayısı
*   monetary_value= satınalma başına düşen ort. kazanç



In [47]:
today_date = dt.datetime(2011, 12, 11) #analiz tarihi
cltv_df = df.groupby('Customer ID').agg(
    {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days,#recency,gün cinsi
                     lambda InvoiceDate: (today_date - InvoiceDate.min()).days],#T, gün cinsi
     'Invoice': lambda Invoice: Invoice.nunique(), #müşteri bazlı toplam fatura sayısı frequency
     'TotalPrice': lambda TotalPrice:TotalPrice.sum()}) #monetary


In [48]:
cltv_df.head()

InvoiceDate             Invoice TotalPrice
             <lambda_0> <lambda_1> <lambda>   <lambda>
Customer ID                                           
12346.0000            0        326        1 77183.6000
12347.0000          365        368        7  4310.0000
12348.0000          282        359        4  1797.2400
12349.0000            0         19        1  1757.5500
12350.0000            0        311        1   334.4000

In [49]:
cltv_df.columns = cltv_df.columns.droplevel(1)

In [50]:
cltv_df.head()

,InvoiceDate,InvoiceDate,Invoice,TotalPrice
Customer ID,,,,
12346.0000,0,326,1,77183.6000
12347.0000,365,368,7,4310.0000
12348.0000,282,359,4,1797.2400
12349.0000,0,19,1,1757.5500
12350.0000,0,311,1,334.4000


In [51]:
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

In [53]:
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,77183.6000
12347.0000,365,368,7,4310.0000
12348.0000,282,359,4,1797.2400
12349.0000,0,19,1,1757.5500
12350.0000,0,311,1,334.4000


In [54]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"] #ortalama harcama toplam harcama/toplam alışveriş yani fatura sayısı

cltv_df = cltv_df[(cltv_df['frequency'] > 1)] #toplam frekans 1 den küçük olamaz çünkü 1 kez alışveriş yapmış kişiler henüz müşteri sayılmıyor 2. kezden sonra dahil ediliyorlar, bu sebeple hesaptan çıkaralım

cltv_df["recency"] = cltv_df["recency"] / 7 # haftalık olmalı

cltv_df["T"] = cltv_df["T"] / 7 #haftalık olmalı

In [59]:
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12347.0000,52.1429,52.5714,7,615.7143
12348.0000,40.2857,51.2857,4,449.3100
12352.0000,37.1429,42.4286,8,313.2550
12356.0000,43.1429,46.5714,3,937.1433
12358.0000,21.2857,21.5714,2,584.0300


In [57]:
bgf = BetaGeoFitter(penalizer_coef=0.001) # fonksiyonu tanımladık coef katsayı ceza oranıdır tolerans oranı

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T']) #fitleme işlemi yaptı tahminledi, alpha,beta ve r bulundu

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 11.41, b: 2.49, r: 2.18>

In [67]:
# 1 aylık en çok satınalma yapacak 10 müşteri tahmini:
bgf.predict(4,                   #haftalığa çevirdik zamanı, 4 hafta
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sort_values(ascending=False).head(10) #azalan olarak dizip ilk 10 u getir



,0
Customer ID,
12748.0000,12.9633
14911.0000,12.4722
17841.0000,7.7398
13089.0000,6.1330
14606.0000,5.8399
15311.0000,5.7191
12971.0000,5.4131
14646.0000,4.8119
13408.0000,3.9341


In [68]:
#dfye ekleyelim
cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

bgf.predict(4,
            cltv_df['frequency'],
            cltv_df['recency'],
            cltv_df['T']).sum()

1776.8934732202908

In [69]:
cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_month,expected_purc_1_month_round,expected_average_profit
Customer ID,,,,,,,
12347.0000,52.1429,52.5714,7,615.7143,0.5635,1,632.0200
12348.0000,40.2857,51.2857,4,449.3100,0.3668,0,470.8035
12352.0000,37.1429,42.4286,8,313.2550,0.7271,1,320.7242
12356.0000,43.1429,46.5714,3,937.1433,0.3435,0,996.4109
12358.0000,21.2857,21.5714,2,584.0300,0.4862,0,641.9691


In [84]:
# 3. GAMMA-GAMMA Modelinin Kurulması

ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df['frequency'], cltv_df['monetary'])



<lifetimes.GammaGammaFitter: fitted with 2845 subjects, p: 3.79, q: 0.34, v: 3.72>

In [83]:
ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).sort_values(ascending=False).head(10)

,0
Customer ID,
16446.0000,92301.0619
15749.0000,15764.0516
15098.0000,14129.6000
12415.0000,5998.3918
12590.0000,5406.2527
12536.0000,4461.6833
16000.0000,4388.0165
18102.0000,4340.3219
12435.0000,4291.7066


In [85]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])
cltv_df.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,expected_purc_1_month,expected_purc_1_month_round,expected_average_profit
Customer ID,,,,,,,
16446.0000,29.1429,29.4286,2,84236.2500,0.3933,0,92301.0619
15749.0000,13.8571,47.5714,3,14844.7667,0.1116,0,15764.0516
15098.0000,0.0000,26.1429,3,13305.5000,0.0401,0,14129.6000
12415.0000,44.7143,48.2857,21,5948.3110,1.5139,2,5998.3918
12590.0000,0.0000,30.2857,2,4932.1300,0.0460,0,5406.2527
12536.0000,2.1429,8.5714,3,4200.6100,0.8559,1,4461.6833
16000.0000,0.0000,0.4286,3,4131.2333,1.6639,2,4388.0165
18102.0000,52.2857,52.5714,60,4327.6217,3.8636,4,4340.3219
12435.0000,26.8571,38.2857,2,3914.9450,0.3041,0,4291.7066


In [86]:
# 4. BG-NBD ve GG modeli ile CLTV'nin hesaplanması.

cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=3,  # 3 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)


In [87]:
cltv = cltv.reset_index()

In [88]:
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,Customer ID,recency,T,frequency,monetary,expected_purc_1_month,expected_purc_1_month_round,expected_average_profit,clv
1970,16446.0000,29.1429,29.4286,2,84236.2500,0.3933,0,92301.0619,114516.9149
1122,14646.0000,50.4286,50.7143,73,3838.4386,4.8119,5,3847.6982,58682.8288
2761,18102.0000,52.2857,52.5714,60,4327.6217,3.8636,4,4340.3219,53160.9468
2458,17450.0000,51.2857,52.5714,46,4229.3650,2.9815,3,4245.5707,40127.8617
843,14096.0000,13.8571,14.5714,17,3833.2229,2.8955,3,3873.2473,35215.1583
36,12415.0000,44.7143,48.2857,21,5948.3110,1.5139,2,5998.3918,28770.0885
1257,14911.0000,53.1429,53.4286,201,715.5476,12.4722,12,716.1886,28321.0209
874,14156.0000,51.5714,53.1429,55,2134.1751,3.5005,4,2141.0438,23760.5466
1754,16000.0000,0.0000,0.4286,3,4131.2333,1.6639,2,4388.0165,22564.5503
2487,17511.0000,52.8571,53.4286,31,2937.4961,2.0298,2,2954.2658,19010.3290


In [89]:
# 5. CLTV'ye Göre Segmentlerin Oluşturulması
cltv_final

cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

In [90]:
cltv_final.sort_values(by="clv", ascending=False).head(10)




,Customer ID,recency,T,frequency,monetary,expected_purc_1_month,expected_purc_1_month_round,expected_average_profit,clv,segment
1970,16446.0000,29.1429,29.4286,2,84236.2500,0.3933,0,92301.0619,114516.9149,A
1122,14646.0000,50.4286,50.7143,73,3838.4386,4.8119,5,3847.6982,58682.8288,A
2761,18102.0000,52.2857,52.5714,60,4327.6217,3.8636,4,4340.3219,53160.9468,A
2458,17450.0000,51.2857,52.5714,46,4229.3650,2.9815,3,4245.5707,40127.8617,A
843,14096.0000,13.8571,14.5714,17,3833.2229,2.8955,3,3873.2473,35215.1583,A
36,12415.0000,44.7143,48.2857,21,5948.3110,1.5139,2,5998.3918,28770.0885,A
1257,14911.0000,53.1429,53.4286,201,715.5476,12.4722,12,716.1886,28321.0209,A
874,14156.0000,51.5714,53.1429,55,2134.1751,3.5005,4,2141.0438,23760.5466,A
1754,16000.0000,0.0000,0.4286,3,4131.2333,1.6639,2,4388.0165,22564.5503,A
2487,17511.0000,52.8571,53.4286,31,2937.4961,2.0298,2,2954.2658,19010.3290,A


In [91]:
cltv_final.groupby("segment").agg(
    {"count", "mean", "sum"})


Customer ID                     recency                        T               frequency                  monetary                expected_purc_1_month              expected_purc_1_month_round              expected_average_profit                         clv                
                  sum       mean count        sum    mean count        sum    mean count       sum    mean count         sum     mean count                   sum   mean count                         sum   mean count                     sum     mean count          sum      mean count
segment                                                                                                                                                                                                                                                                                    
D       11079720.0000 15561.4045   712 15761.1429 22.1364   712 28767.7143 40.4041   712      2181  3.0632   712 132140.0065 185.5899   712              202.8510 0.2849   712                          83 0.1166   712             143308.7036 201.2763   712  104286.7681  146.4702   712
C       10892305.0000 15319.6976   711 21876.4286 30.7685   711 27130.1429 38.1577   711      2923  4.1111   711 195886.5152 275.5085   711              342.6580 0.4819   711                         282 0.3966   711             209131.5403 294.1372   711  274766.4382  386.4507   711
B       10889005.0000 15315.0563   711 20759.8571 29.1981   711 24621.2857 34.6291   711      3863  5.4332   711 270217.4314 380.0526   711              459.6450 0.6465   711                         481 0.6765   711             285206.7020 401.1346   711  495811.4270  697.3438   711
A       10645181.0000 14972.1252   711 22443.4286 31.5660   711 24658.1429 34.6809   711      8072 11.3530   711 603792.1795 849.2154   711              771.7394 1.0854   711                         801 1.1266   711             633105.4391 890.4437   711 1800018.6123 2531.6717   711